In [ ]:
import cv2
import numpy as np
import os
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Body parts and pose pairs definition (COCO Model)
BODY_PARTS = {"Nose": 0, "Neck": 1, "RShoulder": 2, "RElbow": 3, "RWrist": 4,
              "LShoulder": 5, "LElbow": 6, "LWrist": 7, "RHip": 8, "RKnee": 9,
              "RAnkle": 10, "LHip": 11, "LKnee": 12, "LAnkle": 13, "REye": 14,
              "LEye": 15, "REar": 16, "LEar": 17, "Background": 18}


image_dir = 'traffic'

# Load the neural network model
net = cv2.dnn.readNetFromTensorflow("/Users/melvindennies/Desktop/thesis_test/graph_opt.pb")

def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    ba = a - b
    bc = c - b
    if np.linalg.norm(ba) == 0 or np.linalg.norm(bc) == 0:
        return 0
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    cosine_angle = np.clip(cosine_angle, -1.0, 1.0)
    angle = np.arccos(cosine_angle)
    return np.degrees(angle)

def calculate_distance(a, b):
    a = np.array(a)
    b = np.array(b)
    return np.linalg.norm(a - b)

def extract_pose_features(image_path):
    frame = cv2.imread(image_path)
    if frame is None:
        print(f"Error loading image: {image_path}")
        return None
    frameWidth = frame.shape[1]
    frameHeight = frame.shape[0]
    inWidth = 368
    inHeight = 368

    inp = cv2.dnn.blobFromImage(frame, 1.0, (inWidth, inHeight),
                               (127.5, 127.5, 127.5), swapRB=True, crop=False)
    net.setInput(inp)
    out = net.forward()
    out = out[:, :19, :, :]

    points = []
    threshold = 0.1
    for i in range(len(BODY_PARTS)):
        heatMap = out[0, i, :, :]
        _, conf, _, point = cv2.minMaxLoc(heatMap)
        x = (frameWidth * point[0]) / out.shape[3]
        y = (frameHeight * point[1]) / out.shape[2]
        if conf > threshold:
            points.append((int(x), int(y)))
        else:
            points.append(None)

    features = []

    angle_joints = [
        ("Neck", "RShoulder", "RElbow"),
        ("RShoulder", "RElbow", "RWrist"),
        ("Neck", "LShoulder", "LElbow"),
        ("LShoulder", "LElbow", "LWrist"),
        ("RHip", "RKnee", "RAnkle"),
        ("LHip", "LKnee", "LAnkle"),
        ("Neck", "RHip", "RKnee"),
        ("Neck", "LHip", "LKnee"),
        ("RShoulder", "Neck", "LShoulder"),
        ("RHip", "Neck", "LHip"),
    ]

    for joint1, joint2, joint3 in angle_joints:
        idx1, idx2, idx3 = BODY_PARTS[joint1], BODY_PARTS[joint2], BODY_PARTS[joint3]
        if points[idx1] and points[idx2] and points[idx3]:
            angle = calculate_angle(points[idx1], points[idx2], points[idx3])
            features.append(angle)
        else:
            features.append(0)

    keypoint_pairs = [
        ("RWrist", "LWrist"),
        ("RWrist", "Neck"),
        ("LWrist", "Neck"),
        ("Neck", "Nose"),
        ("Nose", "REye"),
        ("Nose", "LEye"),
    ]

    for kp1, kp2 in keypoint_pairs:
        idx1, idx2 = BODY_PARTS[kp1], BODY_PARTS[kp2]
        if points[idx1] and points[idx2]:
            distance = calculate_distance(points[idx1], points[idx2])
            features.append(distance)
        else:
            features.append(0)

    if points[BODY_PARTS["Neck"]] and points[BODY_PARTS["RHip"]] and points[BODY_PARTS["LHip"]]:
        mid_hip = ((points[BODY_PARTS["RHip"]][0] + points[BODY_PARTS["LHip"]][0]) / 2,
                   (points[BODY_PARTS["RHip"]][1] + points[BODY_PARTS["LHip"]][1]) / 2)
        body_height = calculate_distance(points[BODY_PARTS["Neck"]], mid_hip)
    else:
        body_height = 1

    for idx in range(len(BODY_PARTS)):
        if points[idx] and points[BODY_PARTS["Neck"]]:
            relative_x = (points[idx][0] - points[BODY_PARTS["Neck"]][0]) / body_height
            relative_y = (points[idx][1] - points[BODY_PARTS["Neck"]][1]) / body_height
            features.extend([relative_x, relative_y])
        else:
            features.extend([0, 0])

    return features

def augment_image_multiple(image, image_name, save_dir):
    augmented_images = []
    augmented_image_names = []

    augmented_images.append(image)
    augmented_image_names.append(f"{image_name}_original.jpg")

    brightness_value = 1.3
    bright_image = cv2.convertScaleAbs(image, alpha=brightness_value, beta=0)
    augmented_images.append(bright_image)
    augmented_image_names.append(f"{image_name}_bright.jpg")

    contrast_value = 1.5
    contrast_image = cv2.convertScaleAbs(image, alpha=contrast_value, beta=50)
    augmented_images.append(contrast_image)
    augmented_image_names.append(f"{image_name}_contrast.jpg")

    angle = 15  # Smaller angle to avoid distortion of gesture
    h, w = image.shape[:2]
    M = cv2.getRotationMatrix2D((w // 2, h // 2), angle, 1)
    rotated_image = cv2.warpAffine(image, M, (w, h))
    augmented_images.append(rotated_image)
    augmented_image_names.append(f"{image_name}_rotated.jpg")

    blurred_image = cv2.GaussianBlur(image, (5, 5), 0)
    augmented_images.append(blurred_image)
    augmented_image_names.append(f"{image_name}_blurred.jpg")

    noise = np.random.randn(*image.shape) * 10
    noisy_image = np.clip(image + noise, 0, 255).astype(np.uint8)
    augmented_images.append(noisy_image)
    augmented_image_names.append(f"{image_name}_noisy.jpg")

    for aug_img, aug_img_name in zip(augmented_images, augmented_image_names):
        save_path = os.path.join(save_dir, aug_img_name)
        cv2.imwrite(save_path, aug_img)
        print(f"Saved: {save_path}")

def augment_dataset(image_dir, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for folder_name in os.listdir(image_dir):
        folder_path = os.path.join(image_dir, folder_name)

        if not os.path.isdir(folder_path):
            continue

        output_folder = os.path.join(output_dir, folder_name)

        if not os.path.exists(output_folder):
            os.makedirs(output_folder)

        for image_file in os.listdir(folder_path):
            if image_file.endswith('.jpg') or image_file.endswith('.png'):
                image_path = os.path.join(folder_path, image_file)
                image = cv2.imread(image_path)
                if image is not None:
                    image_name, _ = os.path.splitext(image_file)
                    augment_image_multiple(image, image_name, output_folder)

augment_dataset(image_dir, 'Traffic-Police-Gesture-Recognition-Augmented')

def load_images_from_folders(augmented=False):
    """
    Loads images and corresponding labels (folder names) from the specified directory.
    Filters out non-image files like .DS_Store, and ensures all class images are considered.
    Returns a tuple of (image paths, labels).
    """
    dir_to_use = output_dir if augmented else image_dir
    all_features = []
    all_labels = []
    label_names = []

    for label_name in os.listdir(dir_to_use):
        label_path = os.path.join(dir_to_use, label_name)
        
   
        if not os.path.isdir(label_path) or label_name == ".DS_Store":
            continue

        # Make sure the class label is only added once for each folder
        if label_name not in label_names:
            label_names.append(label_name)
        
        # Iterate through each image file in the label folder
        for image_file in os.listdir(label_path):
        
            if image_file == ".DS_Store":
                continue
            
            if image_file.endswith('.jpg') or image_file.endswith('.png'):
                image_path = os.path.join(label_path, image_file)
                features = extract_pose_features(image_path)
                if features:
                    all_features.append(features)
                    all_labels.append(label_name)  # Ensure correct label is added for each image

    return np.array(all_features), np.array(all_labels), label_names

def classify_gestures_with_confusion_matrix(classifier, augmented=False):
    features, labels, label_names = load_images_from_folders(augmented=augmented)
    
    label_to_int = {label: idx for idx, label in enumerate(label_names)}
    int_labels = np.array([label_to_int[label] for label in labels])

    X_train, X_test, y_train, y_test = train_test_split(features, int_labels, test_size=0.2, random_state=42)

    clf = classifier
    clf.fit(X_train, y_train)

    y_pred = clf.predict(X_test)
    
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Classification accuracy: {accuracy:.2f}")

    cm = confusion_matrix(y_test, y_pred)
    
    plt.figure(figsize=(10, 7))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=label_names, yticklabels=label_names)
    plt.title('Confusion Matrix for Gesture Classification')
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.show()

    pca = PCA(n_components=2)
    reduced_features = pca.fit_transform(features)
    plt.figure(figsize=(10, 7))
    scatter = plt.scatter(reduced_features[:, 0], reduced_features[:, 1], c=int_labels, cmap='viridis', alpha=0.7)
    plt.title("Gesture Classification Feature Space")
    plt.xlabel("Principal Component 1")
    plt.ylabel("Principal Component 2")
    plt.colorbar(scatter, label='Gesture Type')
    plt.show()

# Example usage with different classifiers
print("Random Forest Classifier:")
classify_gestures_with_confusion_matrix(RandomForestClassifier(n_estimators=100, random_state=42), augmented=True)

print("Support Vector Classifier (SVM):")
classify_gestures_with_confusion_matrix(SVC(kernel='linear'), augmented=True)

print("k-Nearest Neighbors (k-NN):")
classify_gestures_with_confusion_matrix(KNeighborsClassifier(n_neighbors=5), augmented=True)

print("Gradient Boosting Classifier:")
classify_gestures_with_confusion_matrix(GradientBoostingClassifier(n_estimators=100, random_state=42), augmented=True)
